In [34]:
import arcpy
from archaic import FeatureClass

arcpy.env.workspace = ".data/canada.geodatabase"  # type: ignore

class Mine:
    objectid: int
    name_e: str
    type_e: str
    shape: arcpy.PointGeometry

feature_class = FeatureClass[Mine]("mines_pt")

for mine in feature_class.read(wkid=4326):
    print(mine.name_e, mine.shape.WKT)


Victor POINT (-83.895934386762889 52.821169587240128)
Musselwhite POINT (-90.391753337661598 52.597857399398237)
McArthur River POINT (-105.07011707097534 57.75130033993338)
Raglan POINT (-73.617715175947467 61.709178402138583)
Gibraltar POINT (-122.27393333756686 52.530587668270641)
Bachelor Lake POINT (-76.125962612264829 49.479163744275617)
Bucko Lake POINT (-98.658512016764647 54.834410783991956)
Meadowbank POINT (-95.975977711453567 65.024794972817887)
Ekati POINT (-110.59001135562784 64.761306673101075)
Snap Lake POINT (-110.76748173732163 63.636445658241044)
Diavik POINT (-110.28191553739606 64.492831064536588)
Chisel North POINT (-100.14142376007227 54.86226172896982)
Seabee POINT (-103.64582850214336 55.708761144796192)
McClean Lake POINT (-103.80222109816 58.235748059881303)
Key Lake POINT (-105.66176925926359 57.263924248948761)
Rabbit Lake POINT (-103.68718028354095 58.218593246598708)
Whitewood POINT (-114.52163258505115 53.645427207566868)
Genesee POINT (-114.252843296882